### Import DataBase

In [ ]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

 - id - id сообщения в ТГ 
 - 1   type - тип действия: 
    - message (сообщение)
    - service (звонки)
 - 2   date - дата написания сообщения (с 1 марта 2023 до 18 декабря 2023)
 - 3   date_unixtime - дата сообщения в виде временной метки (удалить)
 - 4   from - от кого сообщение
 - 5   from_id - ID пользователя от которого пришло сообщение (Удалить)
 - 6   text/0 - только текст сообщения
 - 7   text_entities/0/type - тип сообщения: 
    - plain (обычное) 
    - link (ссылка)
 - 8   text_entities/0/text/0 - и текст и ссылки
 - 9   forwarded_from - от кого переслано сообщение
 - 10  photo - ссылка на фото
 - 11  width - ширина носителя (удалить)
 - 12  height - высота носителя (удалить)
 - 13  reply_to_message_id - Если сообщение является ответом, укажите ID исходного сообщения 
 - 14  edited - дата редактирования сообщения 
 - 15  edited_unixtime - дата редактирования сообщения в виде временной метки (удалить)
 - 16  actor - имя пользователя, который совершает действие (звонок например)
 - 17  actor_id - ID пользователя, который совершает действие (звонок например)
 - 18  action - действие (phone_call звонок)
 - 19  duration_seconds - время звонка
 - 20  discard_reason:
    - busy - занято
    - disconnect - отключено
    - hangup - принят
    - missed - пропущено
 - 21  file - ссылка на файл (удалить) 
 - 22  mime_type - тип мультимедии:
    - application/x-navistyle
    - video/mp4 (видео)
    - image/png (фото)
 - 23  text/0/type - тип сообщения:
    - link - ссылка 
 - 24  text/0/text/0 - ссылка
 - 25  text/1/type             1 non-null      object 
 - 26  text/1/text/0           1 non-null      object 
 - 27  text/2 - (удалить)
 - 28  text_entities/1/type - тип подписи (закинуть как отдельно сообщение)
 - 29  text_entities/1/text/0 - если ссылку или фото подписать то текст пойдет сюда (закинуть как отдельно сообщение)
 - 30  text_entities/2/type - тип подписи (закинуть как отдельно сообщение)
 - 31  text_entities/2/text/0 - если ссылку или фото подписать то текст пойдет сюда (закинуть как отдельно сообщение)
 - 32  thumbnail - (удалить)
 - 33  media_type - тип сообщения:
    - sticker
    - video_message
    - voice_message
    - animation
    - video_file
    - audio_file
 - 34  text/1 - если ссылку или фото подписать то текст пойдет сюда (закинуть как отдельно сообщение)

In [ ]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [ ]:
name="Amalia"
df=pd.read_csv(f"./chats/{name}/Chat_{name}.csv")

### Functions

In [ ]:
def create_profile_report(df, name="Report"):
    profile = ProfileReport(df, title=str(name))
    profile.to_notebook_iframe()
    profile.to_file(output_file=f"./chats/{name}/Report_{name}.html")

In [ ]:
def create_new_rows(df):
    for index, row in df.iterrows():
        main_text=row['text/0']
        main_type=row['text_entities/0/type']
        
        columns={
            # row['text_entities/0/text/0']:main_type,
            # row['text/0/text/0']:row["text/0/type"],
            # row['text/1/text/0']:row["text/1/type"],
            # row['text_entities/1/text/0']:row["text_entities/1/type"],
            # row['text_entities/2/text/0']:row["text_entities/2/type"],
            row["text"]:main_type
        }
        array=set([main_text, 
                   row["text"],
                #    row['text_entities/0/text/0'], 
                #    row['text/0/text/0'], 
                #    row['text/1/text/0'], 
                #    row['text_entities/1/text/0'], 
                #    row['text_entities/2/text/0']
                ])
        array.remove(main_text)
        for i in array:
            if type(i)==str:
                new_row=row.copy()
                new_row["text/0"]=i
                new_row["text_entities/0/type"]=columns[i]
                df.loc[len(df.index)]=new_row
    
    return df

In [ ]:
def replace_missing(df=pd.DataFrame()):
    df.fillna(value="Missing", inplace=True)
    return df

In [ ]:
def message_len(mes):
    if mes=="Missing":
        return 0
    else:
        return len(mes)

In [ ]:
def preprocessing(df):
    df=create_new_rows(df)
    df=replace_missing(df)
    df.drop_duplicates(df)
    df["len_mes"]=df["text/0"].apply(message_len)
    df['date'] = pd.to_datetime(df['date'])
    if {'actor', 'action', "duration_seconds", "discard_reason"}.issubset(df.columns):
        df=df[["id", "type", "date", "from", "text/0",	"text_entities/0/type", "len_mes",	"forwarded_from",	"photo", "reply_to_message_id", "actor", "action", "duration_seconds",	"discard_reason",	"mime_type",	"media_type"]]
    else:
        df=df[["id", "type", "date", "from", "text/0",	"text_entities/0/type", "len_mes",	"forwarded_from",	"photo", "reply_to_message_id", "mime_type","media_type"]]

    df.sort_values(by=["date"])
    return df

In [ ]:
df=preprocessing(df)

### Messages count

In [ ]:
user_mes=df.loc[df["from"]!="Missing", "from"]
users=user_mes.unique()
umc = df.groupby(user_mes).size().reset_index(name='message')

In [ ]:
all_mes=umc["message"].sum()
us1=round((umc.iloc[0]["message"])/all_mes*100)
us2=round((umc.iloc[1]["message"])/all_mes*100)
umc["proc"]=[us1, us2 ]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(y=umc['message'],
                     x=umc["from"],
                     marker=dict(color=["rgb(99, 110, 250)", "rgb(222, 0, 105)"])))
fig.update_layout(title="Count of users messanges")

fig.show()
# fig.write_image("./chats/{name}/graph/'Count_of_users_messanges.png")

In [ ]:
fig = px.pie(umc, values='proc', names='from')
fig.update_layout(width=700, height=700,
                  title="Count of users messanges in percent")

fig.show()
# fig.write_image("./chats/{name}/graph/'Count_of_users_messanges_in_percent.png")

### Activity diogram by days

In [ ]:
# Считаем сколько сообщений за даты
dmc = df.groupby(df['date'].dt.date).size().reset_index(name='message_count')

In [ ]:
fig = go.Figure([go.Scatter(x=dmc['date'], y=dmc['message_count'])])

fig.update_layout(
    title='Activity graph by days',
    xaxis=dict(title='Dates'),
    yaxis=dict(title='Message Count'),
)

fig.show()
# fig.write_image("./chats/{name}/graph/'Activity_by_days.png")

### Activity diogram by hours

In [ ]:
def format_hour(hour):
    print(hour, type(hour))
    return f"{hour:02d}:00"

In [ ]:
hmc = df.groupby(df['date'].dt.hour).size().reset_index(name='message_count')
hmc["date"]=hmc["date"].apply(format_hour)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=hmc['date'], 
               y=hmc['message_count'],
                marker=dict(color="rgb(0, 135, 219)", size=10),
                name="Message count")
)

fig.update_layout(
    shapes=[dict(
            type='rect',
            xref='x',
            yref='paper',
            x0="12:00",
            x1="23:00",
            y0=0,
            y1=1,
            fillcolor='rgb(122, 204, 255)',
            layer='below',
            line=dict(width=0)),
        dict(
            type='rect',
            xref='x',
            yref='paper',
            x0="01:00",
            x1="12:00",
            y0=0,
            y1=1,
            fillcolor='rgb(218, 241, 255)',
            layer='below',
            line=dict(width=0)),
    ]
)

fig.update_layout(
    title='Activity graph by hour',
    xaxis=dict(title='Time'),
    yaxis=dict(title='Message Count'),
)

fig.show()
# fig.write_image("./chats/{name}/graph/'Activity_by_hour.png")

### Mean len of messange

In [ ]:
only_text=df.loc[df["len_mes"]!=0, "len_mes"]
mean_len=round(only_text.mean())

In [ ]:
lmc = df.groupby(only_text).size().reset_index(name='len_count')

In [ ]:
# самое первое сообщение длинной 1200+ символов, чтобы график сильно не растягивался, ограничил его до 100
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=lmc['len_mes'], 
               y=lmc['len_count'],
               name="Сharacter count")
)

fig.add_trace(go.Scatter(x=lmc['len_mes'],
                         y=np.full(len(lmc['len_mes']), mean_len),
                         name="Mean count"))

fig.update_layout(
    title='Message character count',
    xaxis=dict(title='Message count'),
    yaxis=dict(title='Сharacter count'),
    xaxis_range=[0,100]
)

fig.show()
# fig.write_image("./chats/{name}/graph/'Message_character_count.png")

In [ ]:
create_profile_report(df=df, name=name)

### Work with clusters

In [ ]:
df_ml=df[["text/0", "from"]]
df_ml=df_ml.rename(columns={"text/0":"text"})

In [ ]:
df_ml=df_ml[df_ml.text!= "Missing" ]

In [ ]:
df_ml.to_csv (f'./chats/{name}/ml_{name}.csv', index= False )